In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dados.csv')

In [3]:
df.head()

,titulo,autor,ISBN_13,ISBN_10,ano,paginas,idioma,editora,rating,avaliacao,resenha,abandonos,relendo,querem_ler,lendo,leram,descricao,genero,male,female
0,Orçamento sem falhas,Nath Finanças,9.786556e+12,6555601566,2021,128,português,Intrínseca,0.0,0,0,0,0,1,0,0,"Quando o assunto é dinheiro, não dá para achar...","Economia, Finanças / Literatura Brasileira / N...",28,72
1,Minha Sombria Vanessa,Kate Elizabeth Russell,9.788551e+12,8551006274,2020,432,português,Intrínseca,4.3,2,653,48,3,6,389,2,Elogiado por Gillian Flynn e considerado um do...,Drama / Ficção / Literatura Estrangeira / Susp...,7,93
2,Recursão,Blake Crouch,9.788551e+12,8551005375,2020,320,português,Intrínseca,4.4,2,618,22,2,6,295,2,E se um dia memórias vívidas de coisas que nun...,Ficção / Ficção científica / Literatura Estran...,21,79
3,"M, o Filho do Século",Antonio Scurati,9.788551e+12,855100607X,2020,816,português,Intrínseca,4.6,60,12,1,0,951,49,55,"O romance M, o Filho do Século conta em trama ...","Biografia, Autobiografia, Memórias / História ...",37,63
4,Oblivion Song: Entre Dois Mundos,Robert Kirkman,9.788551e+12,8551006258,2020,136,português,Intrínseca,4.1,166,16,0,0,376,6,184,Mestre em traçar universos distópicos permeado...,"Ficção / HQ, comics, mangá / Literatura Estran...",44,56


In [4]:
df.isnull().sum() #Verificando valores nulos

titulo          0
autor           4
ISBN_13       759
ISBN_10       759
ano             0
paginas         0
idioma          2
editora         1
rating          0
avaliacao       0
resenha         0
abandonos       0
relendo         0
querem_ler      0
lendo           0
leram           0
descricao     657
genero        589
male            0
female          0
dtype: int64

In [5]:
df = df.drop('ISBN_13', axis=1) #Excluindo a coluna 'ISBN_13'

In [6]:
df = df.drop('ISBN_10', axis=1) #Excluindo a coluna 'ISBN_10'

In [7]:
df['descricao'] = df['descricao'].fillna('Sem descrição') #Preenchendo os valores nulos da coluna 'descricao'

In [8]:
df['genero'] = df['genero'].fillna('Não especificado') #Preenchendo os valores nulos da coluna 'genero'

In [9]:
pip install nltk #Instalando o nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Pegando as stopwords em português
stopwords_pt = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/erikamaschio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# Inicializando o TF-IDF com as stopwords em português
tfidf = TfidfVectorizer(stop_words=stopwords_pt)

In [13]:
# Aplicando o TF-IDF nas descrições dos livros
tfidf_matrix = tfidf.fit_transform(df['descricao'])

In [14]:
# Exibindo o formato da matriz TF-IDF
print(tfidf_matrix.shape)  # Quantas linhas (livros) e colunas (palavras) temos

(11975, 75560)


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Calculando a similaridade entre os livros
cosine_sim = cosine_similarity(tfidf_matrix)
print(cosine_sim)

[[1.         0.0206894  0.00982278 ... 0.00759234 0.00669794 0.        ]
 [0.0206894  1.         0.01549611 ... 0.01079401 0.02968223 0.00612293]
 [0.00982278 0.01549611 1.         ... 0.01428103 0.01777482 0.00618177]
 ...
 [0.00759234 0.01079401 0.01428103 ... 1.         0.02001364 0.0197798 ]
 [0.00669794 0.02968223 0.01777482 ... 0.02001364 1.         0.0145546 ]
 [0.         0.00612293 0.00618177 ... 0.0197798  0.0145546  1.        ]]


In [17]:
def recomendar_livros(nome_livro, df, cosine_sim):
    # Encontrando o índice do livro com base no nome
    idx = df[df['titulo'].str.lower() == nome_livro.lower()].index[0]

    # Calculando a similaridade entre o livro escolhido e todos os outros
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #Ordenando os livros por similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Excluindo o próprio livro da lista
    sim_scores = sim_scores[1:3]
    
    #Obtendo os títulos dos livros recomendados
    livros_recomendados = [df['titulo'][i[0]] for i in sim_scores]
    
    return livros_recomendados